In [ ]:
import os
import json
import cv2
import numpy as np
import os
import sys
import math
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from random import randrange

IMG_WIDTH = 224
IMG_HEIGHT = 224
NORMALIZE = True
TEST_SIZE = 500
READ_SIZE = 10000
#Das garantiert nicht eine Test size, sondern ist nur ein maximum für die test size, solange nicht alle bilder entzippt wurden

# number of learnings should be until the loss converges otherwise we overfit
EPOCHS = 50
TOTAL_EPOCHS = EPOCHS + 300
NUM_CATEGORIES = 3
# drop out nodes at random to make the network more robust
DROPOUT = 0.2
HIDDENLAYERS_SIZE = 64
STEPS_PER_EPOCH = 1
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
IMG_SHAPE = (IMG_HEIGHT,IMG_WIDTH,3)
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

INSTA_DATA = '/storage/insta_data/insta_data/'
IMAGES_DIR = INSTA_DATA+'Post images (JPG files)/images/'
INFO_DIR = INSTA_DATA+'Post metadata (JSON files)/inof/'
org_stdout = sys.stdout
image_mapping = {}
info_mapping = {}
print("Test")
print(sys.stdout)
print(org_stdout)

In [ ]:
 with open(INSTA_DATA+"influencers.txt",'r') as f:
        f.readline()
        f.readline()
        line = f.readline().split("\t")
        while len(line) == 5:
            info_mapping[line[0]] = {
                'category':line[1],
                'followers':line[2],
                "following":line[3],
                'posts':line[4].rstrip()
            }
            line = f.readline().split("\t")

In [ ]:


def image_train(skip = 0,till=-1,Fast=True,batch_size=32):
    if Fast:
        out_images = []
        out_labels = []
        #hier werden jeweils Bild-Metadaten paare gespeichert, damit diese dann eingelesen werden können.
        with open("pairs_w_dupes.txt","r") as pi:
            for i in range(skip):
                next(pi)
            lines_read = skip+1
            l = pi.readline()[63:].rstrip()
            while not l == '' and (lines_read < skip+till or till < 0):
                if len(out_images) == batch_size:
                    yield np.array([i for i in out_images]), np.array([i for i in out_labels])
                    out_labels = []
                    out_images = []
                info_file = INFO_DIR+l
                image_file = IMAGES_DIR+l.replace(".info",".jpg")
                influencer = l[:l.rfind("-")]
                if (os.path.exists(image_file)) and influencer in info_mapping:
                    followers = int(info_mapping[influencer]['followers'])
                    following = int(info_mapping[influencer]['following'])
                    info = {}
                    with open(info_file,'r') as info_file_object:
                        info = json.loads(info_file_object.read())
                        like = int(info["edge_media_preview_like"]["count"])
                    image = cv2.imread(image_file)
                    resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),interpolation=cv2.INTER_AREA)
                    # make the input better for the neural network
                    if(NORMALIZE):
                        resized = resized / 255.0
                    #print('shape '+resized.shape())
                    out_images.append(resized)
                    out_labels.append(like)
                else:
                    print('CRITICAL ERROR')
                    print(image_file)
                l = pi.readline()[63:].rstrip()
                lines_read += 1
        max_like = max(out_labels)
        var = np.var(out_labels)
        mean = np.mean(out_labels)
        sqr_var = math.sqrt(var)
        yield np.array([i for i in out_images]), np.array([i/max_like for i in out_labels])
    else:
      #hier kann nach neuen Bild-Metadaten Paaren gesucht werden, die dann auch gespeichert werden (können)
        with open(INSTA_DATA+"JSON-Image_files_mapping.txt",'r') as f:
            print(skip)
            print(till)
            f.readline()
            for _ in range(skip):
                next(f)
            line = f.readline().split("\t")
            line[2].rstrip()
            '''with open("pairs.txt","w+") as p:
                p.write('')
                print('cleared pairs')'''
            readline = 2+skip
            one_image_posts = 0
            info_file = ""
            image_str = ""
            image_file = ""
            infos = 0
            images = 0
            out = []
            out_images = []
            out_labels = []
            while len(line) == 3 and (readline < till or till<0):
                if len(out_images) == batch_size:
                    yield np.array([i for i in out_images]), np.array([i for i in out_labels])
                    out_labels = []
                    out_images = []
                line[2] = line[2].rstrip()
                strs = line[2][2:-2].split(',')
                if len(strs) == 1:
                    one_image_posts += 1
                    info_file = INFO_DIR+line[0]+'-'+line[1][:-5]+'.info'
                    image_str = strs[0].lstrip().replace("'","")
                    image_file = IMAGES_DIR+line[0]+'-'+image_str
                    if os.path.exists(info_file):
                        infos+=1
                    if(os.path.exists(image_file)):
                        images+=1
                    if os.path.exists(info_file) and os.path.exists(image_file):
                        print('YAY')
                        followers = int(info_mapping[line[0]]['followers'])
                        following = int(info_mapping[line[0]]['following'])
                        info = {}
                        with open(info_file,'r') as info_file_object:
                            info = json.loads(info_file_object.read())
                            print(info.keys())
                            like = int(info["edge_media_preview_like"]["count"])
                            #print(info['likes']
                        print(image_file)
                        image = cv2.imread(image_file)
                        resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),interpolation=cv2.INTER_AREA)
                        # make the input better for the neural network
                        if(NORMALIZE):
                            resized = resized / 255.0
                        print(str(infos) +' '+str(images))
                        with open('/storage/pairs.txt','a+') as pa:
                            pa.write(info_file+'\n')
                        #print('shape '+resized.shape())
                        out_images.append(resized)
                        out_labels.append((like-following)/(followers-following))
                            
                line = f.readline().split("\t")
                readline += 1
                if readline%10000 == 0:
                    line[2] = line[2].rstrip()
                    strs = line[2][2:-2].split(',')
                    print(line)
                    print(readline)
                    print(strs)
                    print(line[2])
                    print(one_image_posts)
                    print(str(os.path.exists(info_file))+info_file)
                    print(str(os.path.exists(image_file))+image_file)
                    print(str(images))
                    print(str(infos))
#add tensorflow methods and shape of dataset (see x_train)


x_train,y_train = next(image_train(skip=TEST_SIZE,till=TEST_SIZE+READ_SIZE,batch_size=READ_SIZE))
x_test,y_test = next(image_train(till=TEST_SIZE,batch_size=TEST_SIZE))

In [ ]:


base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                  include_top=False,
                                               weights='imagenet')
LEARNING_RATE = 0.001
# batchsize = x -> could define one // woundn't use it except if it takes to long
# it is the amount of images in the data set which 
# is getting looked at before an adjustment


def main():
    txt = sys.stdout
    sys.stdout = org_stdout
    sys.stdout = open('file5.txt', 'w+')
    print(txt)
    print(len(x_train))
    print(len(x_test))
    model = get_model()
    if True: 
      model = tf.keras.models.load_model('/storage/Deep_CNN')

    #grid search
    """
    model = KerasClassifier(build_fn = lambda:get_model())
    
    batch_size = [10, 50, 100]
    epochs = [5, 10 , 20,  50, 100]
    dropout_rate = [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    pool_type =  ['max', 'average']
    param_grid = dict(batch_size = batch_size, epochs = epochs)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3,
                        verbose = 2)
    grid_result = grid.fit(x_train, y_train)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_,
                                 grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    """
    print(TEST_SIZE)
    # Fit model on training data
    History = model.fit(x=x_train,y=y_train, verbose=2,epochs=EPOCHS)

     # Evaluate neural network performance
    model.evaluate(x=x_test,y=y_test, verbose=2)


    # prepare the base model
    base_model.trainable = True

    # Let's take a look to see how many layers are in the base model
    print("Number of layers in the base model: ", len(base_model.layers))

    # Fine-tune from this layer onwards
    fine_tune_at = 100

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable =  False
    model.summary()

    for layer in base_model.layers[fine_tune_at:]:
      if layer.name[-2:] == "BN" or layer.name[-2:] == "bn":
        layer.trainable = False
    model.summary()
    
    decayrate = 1
    
    #finds the last learning rate of the pretrained model
    base_learning_rate = LEARNING_RATE / (1 + decayrate * STEPS_PER_EPOCH /
                                          STEPS_PER_EPOCH*1000)

    #compile the model for finetuning
    model.compile(loss="mean_absolute_percentage_error",
              optimizer = tf.keras.optimizers.RMSprop(
                  learning_rate=base_learning_rate),
              metrics=["mean_absolute_percentage_error"]
              )

    #train the finetuned model with a smaler learningrate
    history_fine =  model.fit(x=x_train, y=y_train,
                         epochs=TOTAL_EPOCHS,
                         initial_epoch=History.epoch[-1],verbose=2)

    # Evaluate neural network performance

    sys.stdout = open('file4.txt', 'w+')
    model.evaluate(x=x_test,y=y_test, verbose=2)
    model.summary()
    
    filename = ("/storage/Deep_CNN")
    model.save(filename)

    # Save model to file
    #if len(sys.argv) == 3:
    #    filename = sys.argv[2]
    #    model.save(filename)
    #    #print(f"Model saved to {filename}.")

def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    # make the model more robust
    data_augmentation = tf.keras.Sequential(
      [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                    input_shape=(IMG_HEIGHT, 
                                                                  IMG_WIDTH,
                                                                  3)),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
      ]
    )

    # decrease the rate of change towards the end
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        LEARNING_RATE,
        decay_steps=STEPS_PER_EPOCH*1000,
        decay_rate=1,
        staircase=False)
    
    # Create the base model from the pre-trained model MobileNet V2
    IMG_SHAPE = IMG_SIZE + (3,)
    base_model.trainable = False
    # bias
    #model with several layers
    model = tf.keras.models.Sequential([
        data_augmentation,                                
        # add the convolution layer -> extract feature such as strong colour difference
        #-> lernt Kanten
        base_model,
        # extract information
        tf.keras.layers.GlobalAveragePooling2D(),
        # add a dropout
        tf.keras.layers.Dropout(DROPOUT),
        # linear classifier as we want a continues value
        tf.keras.layers.Dense(1, activation = "linear")
    ])
    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss="mean_absolute_percentage_error",
        metrics=["mean_absolute_percentage_error"]
    )
    return model

if __name__ == "__main__":
  main()